In [2]:
import  pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler




In [3]:
import sys
import os

# Add scripts path
sys.path.append(os.path.abspath('../scripts'))

In [5]:
from load_data import connect_to_db
from data_cleaning import  clean_numeric_columns, remove_outliers
from data_formatting import format_columns, convert_time_to_hours, format_traffic_data
from data_transform import perform_clustering, aggregate_data, top_n_values


In [ ]:
query = "SELECT * FROM xdr_data;"
df = connect_to_db(query)
if df is not None:
    print("successfuly connected")
else:
    print("failed to connect")

In [ ]:
df

In [ ]:
top_handsets = df['Handset Type'].value_counts().head(10)
print("Top 10 Handsets:\n", top_handsets)


In [ ]:
top_manufacturers = df['Handset Manufacturer'].value_counts().head(3)
print("Top 3 Handset Manufacturers:\n", top_manufacturers)


In [ ]:
for manufacturer in top_manufacturers.index:
    top_handsets_per_manufacturer = df[df['Handset Manufacturer'] == manufacturer]['Handset Type'].value_counts().head(5)
    print(f"Top 5 Handsets for {manufacturer}:\n", top_handsets_per_manufacturer)


In [ ]:
# Analyze User Behavior on Applications Analyze User Behavior on Applications
user_behavior = df.groupby('IMSI').agg({
    'Bearer Id': 'sum',
    'Dur. (ms)': 'sum',
    'Total DL (Bytes)': 'sum',
    'Total UL (Bytes)': 'sum',
})
user_behavior['total_data'] = user_behavior['Total DL (Bytes)'] + user_behavior['Total UL (Bytes)']
print(user_behavior.head())


In [ ]:
# Fill missing values for numeric columns
numeric_columns = df.select_dtypes(include=["number"]).columns
df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].mean())

# Fill missing values for non-numeric columns
non_numeric_columns = df.select_dtypes(exclude=["number"]).columns
df[non_numeric_columns] = df[non_numeric_columns].fillna("Unknown")

# Verify there are no missing values
print(df.isnull().sum())


In [57]:
from data_cleaning import clean_handset_data
df = clean_handset_data(df,"Handset Type")


In [16]:
# Define a function to handle outliers using the IQR method
def handle_outliers(col):
    Q1 = col.quantile(0.25)
    Q3 = col.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return np.clip(col, lower_bound, upper_bound)
numeric_cols = df.select_dtypes(include=["number"]).columns

# Apply outlier treatment for numeric columns
for col in numeric_cols:
    df[col] = handle_outliers(df[col])


In [ ]:
# Summary of data types and unique values
variable_summary = pd.DataFrame({
    "Data Type": df.dtypes,
    "Unique Values": df.nunique()
})
print("Variable Summary:")
print(variable_summary)

# Save this summary to a CSV for presentation purposes
variable_summary.to_csv("variable_summary.csv", index=True)


In [ ]:
#Segment users into top five decile classes based on Dur. (ms) and compute total data for each decile.
df['total_data'] = df['Total DL (Bytes)'] + df['Total UL (Bytes)']
df['duration_decile'] = pd.qcut(df['Dur. (ms)'], 5, labels=False)

# Compute total data per decile
decile_data = df.groupby('duration_decile')['total_data'].sum().reset_index()
print("Decile Data Summary:")
print(decile_data)


In [ ]:
basic_metrics = df.describe()
print("Basic Metrics Summary:")
print(basic_metrics)

# Save to CSV for reporting
basic_metrics.to_csv("basic_metrics.csv")


In [ ]:
#Non-Graphical Univariate Analysis

dispersion_params = df[numeric_cols].agg(['mean', 'median', 'std', 'var', 'min', 'max'])
print("Dispersion Parameters:")
print(dispersion_params)

# Save to CSV for reporting
dispersion_params.to_csv("dispersion_params.csv")


In [ ]:
# Histogram for numeric variables
df[numeric_cols].hist(bins=30, figsize=(20, 15))
plt.suptitle("Histograms for Numeric Variables")
plt.savefig("histograms.png")

# Boxplot for numeric variables
plt.figure(figsize=(15, 8))
sns.boxplot(data=df[numeric_cols])
plt.title("Boxplot for Numeric Variables")


In [ ]:
import numpy as np

# Scatterplot for total data vs applications (with log transform)
app_cols = ['Social Media DL (Bytes)', 'Google DL (Bytes)', 'Email DL (Bytes)',
            'Youtube DL (Bytes)', 'Netflix DL (Bytes)', 'Gaming DL (Bytes)', 'Other DL (Bytes)']

for col in app_cols:
    plt.figure(figsize=(8, 6))
    
    # Log-transform the data (avoid log(0) by adding a small constant)
    log_x = np.log1p(df[col])  # log(1 + x) ensures no issues with zero values
    log_y = np.log1p(df['total_data'])
    
    # Scatterplot with log-transformed data
    sns.scatterplot(x=log_x, y=log_y)
    plt.title(f"Log-Transformed Total Data vs {col}")
    plt.xlabel(f"Log {col}")
    plt.ylabel("Log Total Data")
    
    # Save the plot
    plt.show()


In [ ]:
import numpy as np

# Apply log transformation
df['log_social_media_dl'] = np.log1p(df['Social Media DL (Bytes)'])
df['log_total_data'] = np.log1p(df['total_data'])

plt.figure(figsize=(8, 6))
sns.scatterplot(x=df['log_social_media_dl'], y=df['log_total_data'], alpha=0.5)
plt.title("Scatter Plot (Log-Transformed): Total Data vs Social Media DL")
plt.xlabel("Log(Social Media DL Bytes)")
plt.ylabel("Log(Total Data)")
plt.savefig("log_scatter_social_media_dl.png")


In [ ]:
# Correlation matrix
corr_matrix = df[app_cols].corr()
print("Correlation Matrix:")
print(corr_matrix)

# Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm")
plt.title("Correlation Matrix Heatmap")
plt.savefig("correlation_heatmap.png")


In [ ]:
# Dimensionality Reduction (PCA)
# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[app_cols])

# Perform PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(scaled_data)

# Variance explained by each component
print("Explained Variance Ratio:", pca.explained_variance_ratio_)

# Plot PCA results
plt.figure(figsize=(8, 6))
plt.scatter(pca_result[:, 0], pca_result[:, 1], s=10, alpha=0.6)
plt.title("PCA Results")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.grid(alpha=0.3)


In [ ]:
 # TASK 2

In [ ]:
# Aggregating engagement metrics per user

engagement_metrics = df.groupby('MSISDN/Number').agg({
    'Dur. (ms)': 'sum',  
    'Total DL (Bytes)': 'sum',   
    'Total UL (Bytes)': 'sum',   
    'Bearer Id': 'count'      
}).rename(columns={
    'Dur. (ms)': 'Total_Duration',
    'Total DL (Bytes)': 'Total_DL',
    'Total UL (Bytes)': 'Total_UL',
    'Bearer Id': 'Session_Frequency'
})

# Total traffic (download + upload)
engagement_metrics['Total_Traffic'] = engagement_metrics['Total_DL'] + engagement_metrics['Total_UL']

# Top 10 customers per engagement metric
top_10_duration = engagement_metrics.nlargest(10, 'Total_Duration')
top_10_traffic = engagement_metrics.nlargest(10, 'Total_Traffic')
top_10_frequency = engagement_metrics.nlargest(10, 'Session_Frequency')

print("Top 10 customers by duration:")
print(top_10_duration)

print("Top 10 customers by traffic:")
print(top_10_traffic)

print("Top 10 customers by session frequency:")
print(top_10_frequency)


In [ ]:
# Normalize Metrics and Perform K-Means Clustering

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

# Normalize the metrics
scaler = MinMaxScaler()
engagement_metrics_normalized = scaler.fit_transform(engagement_metrics[['Total_Duration', 'Total_Traffic', 'Session_Frequency']])

# Run k-means (k=3)
kmeans = KMeans(n_clusters=3, random_state=42)
engagement_metrics['Cluster'] = kmeans.fit_predict(engagement_metrics_normalized)

print("Cluster Assignments:")
print(engagement_metrics['Cluster'].value_counts())


In [ ]:
# Compute Min, Max, Average, and Total Metrics for Each Cluster


cluster_metrics = engagement_metrics.groupby('Cluster').agg({
    'Total_Duration': ['min', 'max', 'mean', 'sum'],
    'Total_Traffic': ['min', 'max', 'mean', 'sum'],
    'Session_Frequency': ['min', 'max', 'mean', 'sum']
})

print("Cluster Metrics:")
print(cluster_metrics)


In [ ]:
# Aggregate user total traffic per application
app_cols = ['Social Media DL (Bytes)', 'Google DL (Bytes)', 'Email DL (Bytes)',
            'Youtube DL (Bytes)', 'Netflix DL (Bytes)', 'Gaming DL (Bytes)', 'Other DL (Bytes)']

application_metrics = []  # Store metrics for all applications

for app_col in app_cols:
    # Calculate total traffic for each user (MSISDN/Number) for this application
    app_data = df.groupby('MSISDN/Number').agg({
        app_col: 'sum',  # Aggregate the download traffic for this app
        'Total UL (Bytes)': 'sum'  # Aggregate the upload traffic
    }).rename(columns={
        app_col: 'Total_DL',  # Rename download column
        'Total UL (Bytes)': 'Total_UL'  # Rename upload column
    })
    
    # Compute total traffic (download + upload)
    app_data['Total_Traffic'] = app_data['Total_DL'] + app_data['Total_UL']
    
    # Add application name as a column
    app_data['Application'] = app_col.replace(' DL (Bytes)', '')  # Clean app name
    
    # Append to application_metrics list
    application_metrics.append(app_data)

# Concatenate all application metrics into a single DataFrame
application_metrics = pd.concat(application_metrics)

# Top 10 users per application
top_users_per_app = application_metrics.groupby('Application').apply(
    lambda x: x.nlargest(10, 'Total_Traffic')
)

print("Top 10 users per application:")
print(top_users_per_app)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Get the top 3 applications by total traffic
top_apps = application_metrics.groupby('Application')['Total_Traffic'].sum().nlargest(3).index

# Filter data for top 3 applications
top_app_data = application_metrics[application_metrics['Application'].isin(top_apps)]
top_app_data.to_csv("top_applications.csv", index=False)

# Plot traffic distribution

sns.barplot(x='Application', y='Total_Traffic', data=top_app_data, estimator='sum', ci=None)
plt.title("Top 3 Most Used Applications by Total Traffic")
plt.xlabel("Application")
plt.ylabel("Total Traffic (Bytes)")
plt.show()


In [ ]:

# Elbow method
sse = []
k_range = range(1, 10)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(engagement_metrics_normalized)
    sse.append(kmeans.inertia_)

# Save results to a DataFrame
elbow_data = pd.DataFrame({"Number of Clusters (k)": list(k_range), "SSE": sse})

# Save the DataFrame to a CSV file
elbow_data.to_csv("elbow_method.csv", index=False)
print("Elbow method data saved to elbow_method.csv")

# Plot the elbow curve
plt.figure(figsize=(8, 6))
plt.plot(k_range, sse, marker='o')
plt.title("Elbow Method for Optimal K")
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Sum of Squared Errors (SSE)")
plt.show()


In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Assuming 'clustered_data' contains your features
features = clustered_data[['engagement_score', 'experience_score']]
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Calculate the Elbow Method values
elbow_data = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(features_scaled)
    elbow_data.append({'k': k, 'inertia': kmeans.inertia_})

# Save the results to a CSV file
elbow_df = pd.DataFrame(elbow_data)
elbow_df.to_csv('elbow_method.csv', index=False)
print("Elbow method data saved to 'elbow_method.csv'.")


In [ ]:

# Load data

# Task 3.1 - Aggregation per customer
def aggregate_metrics(df):
    # Fill missing values
    df['TCP DL Retrans. Vol (Bytes)'] = df['TCP DL Retrans. Vol (Bytes)'].fillna(df['TCP DL Retrans. Vol (Bytes)'].mean())
    df['TCP UL Retrans. Vol (Bytes)'] = df['TCP UL Retrans. Vol (Bytes)'].fillna(df['TCP UL Retrans. Vol (Bytes)'].mean())
    df['Avg RTT DL (ms)'] = df['Avg RTT DL (ms)'].fillna(df['Avg RTT DL (ms)'].mean())
    df['Avg RTT UL (ms)'] = df['Avg RTT UL (ms)'].fillna(df['Avg RTT UL (ms)'].mean())
    df['Avg Bearer TP DL (kbps)'] = df['Avg Bearer TP DL (kbps)'].fillna(df['Avg Bearer TP DL (kbps)'].mean())
    df['Avg Bearer TP UL (kbps)'] = df['Avg Bearer TP UL (kbps)'].fillna(df['Avg Bearer TP UL (kbps)'].mean())

    # Aggregate metrics
    aggregated = df.groupby('MSISDN/Number').agg({
        'TCP DL Retrans. Vol (Bytes)': 'mean',
        'TCP UL Retrans. Vol (Bytes)': 'mean',
        'Avg RTT DL (ms)': 'mean',
        'Avg RTT UL (ms)': 'mean',
        'Avg Bearer TP DL (kbps)': 'mean',
        'Avg Bearer TP UL (kbps)': 'mean'
    }).reset_index()

    aggregated['Average TCP Retransmission'] = (
        aggregated['TCP DL Retrans. Vol (Bytes)'] + aggregated['TCP UL Retrans. Vol (Bytes)']) / 2
    aggregated['Average RTT'] = (
        aggregated['Avg RTT DL (ms)'] + aggregated['Avg RTT UL (ms)']) / 2
    aggregated['Average Throughput'] = (
        aggregated['Avg Bearer TP DL (kbps)'] + aggregated['Avg Bearer TP UL (kbps)']) / 2

    return aggregated

aggregated_data = aggregate_metrics(df)

# Task 3.2 - Top, bottom, and most frequent values
def compute_top_bottom_most(df, column):
    top_10 = df[column].nlargest(10)
    bottom_10 = df[column].nsmallest(10)
    most_frequent = df[column].value_counts().head(10)
    return top_10, bottom_10, most_frequent

tcp_top, tcp_bottom, tcp_frequent = compute_top_bottom_most(df, 'TCP DL Retrans. Vol (Bytes)')
rtt_top, rtt_bottom, rtt_frequent = compute_top_bottom_most(df, 'Avg RTT DL (ms)')
throughput_top, throughput_bottom, throughput_frequent = compute_top_bottom_most(df, 'Avg Bearer TP DL (kbps)')

# Task 3.3 - Distribution & interpretation
def plot_distribution(df, column, group_by, title, top_n=10):
    """
    Plots the distribution of a specified column grouped by a category,
    showing only the top N categories based on mean values.

    Parameters:
    - df: DataFrame containing the data.
    - column: The column whose distribution is to be plotted.
    - group_by: The column to group the data by.
    - title: The title of the plot.
    - top_n: The number of top categories to include in the plot (default is 10).
    """
    # Group data by the specified column and compute the mean
    grouped = df.groupby(group_by)[column].mean().dropna()

    # Check if there is data to plot
    if grouped.empty:
        print(f"No data to plot for {column} grouped by {group_by}")
        return

    # Select the top N categories
    top_categories = grouped.nlargest(top_n)

    # Plot the data
    top_categories.sort_values().plot(kind='bar', figsize=(12, 6), title=title)
    plt.xlabel(group_by)
    plt.ylabel(column)
    plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
    plt.tight_layout()
    plt.show()

# Example usage:
plot_distribution(df, 'Avg Bearer TP DL (kbps)', 'Handset Manufacturer', 
                  'Average Throughput per Manufacturer (Top 10)', top_n=10)
plt.savefig("Average Troughput per Manufacurere .png")

plot_distribution(df, 'TCP DL Retrans. Vol (Bytes)', 'Handset Manufacturer', 'Average TCP Retransmission per Manufacturer (Top 10)', top_n=10)
plt.savefig("Average TCP Retransmission per Manufacurere .png")

# Task 3.4 - K-means clustering
def perform_clustering(df):
    # Select relevant features
    features = df[['Average TCP Retransmission', 'Average RTT', 'Average Throughput']]

    # Normalize data
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    # K-means clustering
    kmeans = KMeans(n_clusters=3, random_state=42)
    clusters = kmeans.fit_predict(features_scaled)
    df['Cluster'] = clusters

    return df, kmeans

clustered_data, kmeans_model = perform_clustering(aggregated_data)

# Interpret clusters
for cluster_id in range(3):
    print(f"Cluster {cluster_id}:\n")
    cluster_data = clustered_data[clustered_data['Cluster'] == cluster_id]
    print(cluster_data.describe())
    print("\n---\n")


In [22]:
less_engaged_center = kmeans_model.cluster_centers_[2]  # Example: replace 2 with the correct index
worst_experience_center = kmeans_model.cluster_centers_[1]  # Example: replace 1 with the correct index


In [ ]:
from scipy.spatial.distance import euclidean

def calculate_scores(data, target_center):
    return [euclidean(row, target_center) for row in data]

# Calculate scores
user_features = clustered_data[['Average TCP Retransmission', 'Average RTT', 'Average Throughput']].values
clustered_data['engagement_score'] = calculate_scores(user_features, less_engaged_center)
clustered_data['experience_score'] = calculate_scores(user_features, worst_experience_center)


In [ ]:
clustered_data['satisfaction_score'] = (
    clustered_data['engagement_score'] + clustered_data['experience_score']
) / 2


In [25]:
X = clustered_data[['engagement_score', 'experience_score']]
y = clustered_data['satisfaction_score']


In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = model.predict(X_test)
print("MSE:", mean_squared_error(y_test, y_pred))
print("R2 Score:", r2_score(y_test, y_pred))


In [105]:
kmeans = KMeans(n_clusters=2, random_state=42)
clustered_data['satisfaction_cluster'] = kmeans.fit_predict(
    clustered_data[['engagement_score', 'experience_score']]
)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.scatterplot(
    data=clustered_data,
    x='engagement_score',
    y='experience_score',
    hue='satisfaction_cluster',
    palette='viridis'
)
plt.title("K-Means Clustering on Engagement and Experience Scores")
plt.show()


In [ ]:
cluster_summary = clustered_data.groupby('satisfaction_cluster')[
    ['satisfaction_score', 'experience_score']
].mean()
print(cluster_summary)


In [109]:
# Calculate the satisfaction score as the average of engagement and experience scores
aggregated_data['Satisfaction Score'] = (aggregated_data['engagement_score'] + aggregated_data['experience_score']) / 2

# Sort the customers by Satisfaction Score in descending order
sorted_customers = aggregated_data.sort_values('Satisfaction Score', ascending=False)

# Get the top 10 satisfied customers
top_10_satisfied = sorted_customers.head(10)

# Define final_results (this can contain all customers or only the top 10 depending on your needs)
final_results = sorted_customers  # Or use final_results = top_10_satisfied if you only need top 10


In [ ]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Fetch DB credentials from environment variables
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")

# Create the connection string
connection_string = f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

# Create a SQLAlchemy engine
engine = create_engine(connection_string)

# Export the DataFrame to PostgreSQL
try:
    # Save the entire final_results table
    final_results.to_sql('customer_satisfaction', engine, if_exists='replace', index=False)
    
    # Save only the top 10 satisfied customers
    top_10_satisfied.to_sql('top_satisfied_customers', engine, if_exists='replace', index=False)
    
    print("Data exported successfully to PostgreSQL!")
except Exception as e:
    print(f"An error occurred: {e}")
final_results.to_csv("final_results.csv", index=False)


In [ ]:
#

query = "SELECT * FROM customer_satisfaction;"
sf = connect_to_db(query)
if sf is not None:
    print("successfuly connected")
else:
    print("failed to connect")